In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#! pip install global-chem

In [4]:
import global_chem
import inspect
import pandas as pd
import datamol as dm

def fullname(o):
    if inspect.isclass(o):
        klass = o
    else:
        klass = o.__class__
    module = klass.__module__
    if module == '__builtin__':
        return klass.__name__ # avoid outputs like '__builtin__.str'
    return module + '.' + klass.__name__

In [5]:
def build_data():
    gc = global_chem.GlobalChem()
    gc.build_global_chem_network()
    data = []
    for k, v in gc.__NODES__.items():
        if k != 'global_chem' and k != 'common_regex_patterns':
            full_hierarchy  = fullname(v).rsplit(".", 1)[0] + f".{k}"
            df_smiles = pd.DataFrame.from_dict(v.get_smiles(), orient="index", columns=["smiles"])
            df_smarts = pd.DataFrame.from_dict(v.get_smarts(), orient="index", columns=["smarts"])
            df = df_smiles.join(df_smarts, how="outer")
            df = df.rename_axis("iupac").reset_index()
            df["group"] = k
            df["hierarchy"] = full_hierarchy
            data.append(df)
    return pd.concat(data, ignore_index=True)
    

In [6]:
df = build_data()

In [7]:
df.to_csv("../medchem/data/chemical_groups.csv", index=False)